# Daten einlesen und Transformieren

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

def r2(actual: np.ndarray, predicted: np.ndarray):
    """ R2 Score """
    return r2_score(actual, predicted)

def adjr2(actual: np.ndarray, predicted: np.ndarray, rowcount: int, featurecount: int):
    """ Adjusted R2 Score """
    return 1-(1-r2(actual,predicted))*(rowcount-1)/(rowcount-featurecount)

def add_features(df):
    df['Jahr'] = df['Datum'].dt.year
    df['Monat'] = df['Datum'].dt.month
    df['Wochentag'] = df['Datum'].dt.weekday
    df['Kalenderwoche'] = df['Datum'].dt.isocalendar().week
    df['Tag_im_Jahr'] = df['Datum'].dt.dayofyear
    df['Ist_Wochenende'] = df['Wochentag'].isin([5, 6]).astype(int)
    df['Tag_im_Jahr_sin'] = np.sin(2 * np.pi * df['Tag_im_Jahr'] / 365)
    df['Tag_im_Jahr_cos'] = np.cos(2 * np.pi * df['Tag_im_Jahr'] / 365)
    df['Monat_sin'] = np.sin(2 * np.pi * df['Monat'] / 12)
    df['Monat_cos'] = np.cos(2 * np.pi * df['Monat'] / 12)
    df['Wochentag_sin'] = np.sin(2 * np.pi * df['Wochentag'] / 7)
    df['Wochentag_cos'] = np.cos(2 * np.pi * df['Wochentag'] / 7)
    return df

def preprocess_data(df):
    df['Datum'] = pd.to_datetime(df['Datum'])
    df = add_features(df)
    df['Wettercode'] = df['Wettercode'].astype(str)
    df['KielerWoche'] = df['KielerWoche'].fillna(False).astype('bool')
    return df

# Load datasets

# Basisdatenset
df1 = pd.read_csv('../umsatzdaten_gekuerzt.csv')
# Wetterdaten
df2 = pd.read_csv('../wetter.csv')
# Kielerwocher
df3 = pd.read_csv('../kiwo.csv')
# Feiertage, Brückentage, Ferien
df4 = pd.read_csv('../Feier_Bruecke_Ferien_bis2018.csv')
# Verbraucherpreisindex
df5 = pd.read_csv('../VPI.csv')
# Erzeugerpreisindex
df6 = pd.read_csv('../EPI.csv')
# Heimspiele Holstein Kiel
df7 = pd.read_csv('../Heimspiel_Holstein_Kiel_finle.csv')
# Kieler Umschlag (kleineres Fest in Kiel)
df8 = pd.read_csv('../Kieler_Umschlag_finale.csv')
# Schulferien Litauen
df9 = pd.read_csv('../Schulferien_Litauen_finale.csv')

# Merge datasets
df = df1.merge(df2, on='Datum', how='left')
df = df.merge(df3, on='Datum', how='left')
df = df.merge(df4, on='Datum', how='left')
df = df.merge(df5, on='Datum', how='left')
df = df.merge(df6, on='Datum', how='left')
df = df.merge(df7, on='Datum', how='left')
df = df.merge(df8, on='Datum', how='left')
df = df.merge(df9, on='Datum', how='left')


missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
print(missing_values)
# Preprocess data
df = preprocess_data(df)

# Define feature groups
numeric_features = ['Temperatur', 'Bewoelkung','HeimspielHK', 'KielerUmschlag', 'Windgeschwindigkeit', 'Tag_im_Jahr_sin', 'Tag_im_Jahr_cos', 'Monat_sin', 'Monat_cos', 'Wochentag_sin', 'Wochentag_cos', 'feiertag', 'KielerWoche', 'brueckentag', 'BW', 'BY', 'B', 'BB', 'HB', 'HH', 'HE', 'MV', 'NI', 'NW', 'RP', 'SL', 'SN', 'ST', 'SH', 'TH', 'VPI_brot_getreide', 'VPI_molkerei_ei', 'VPI_speiseoel_fett', 'VPI_Obst', 'VPI_zucker_suesses', 'EPI_teig', 'EPI_wasser', 'EPI_strom', 'EPI_fernwaerme', 'Herbstferien_LI', 'Weihnachtsferien_LI', 'Winterferien_LI', 'Fruehlingsferien_LI', 'Sommerferien_LI']
categorical_features = ['Warengruppe', 'Wettercode', 'Wochentag']



# Define ColumnTransformer
wettercode_categories = [str(i) for i in range(0, 100)] + ['Unbekannt']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), [feature for feature in numeric_features if feature != 'KielerWoche']),
        ('kielerwoche', SimpleImputer(strategy='constant', fill_value=0), ['KielerWoche']),
        ('wettercode', OneHotEncoder(categories=[wettercode_categories], handle_unknown='ignore'), ['Wettercode']),
        ('warengruppe', OneHotEncoder(handle_unknown='ignore'), ['Warengruppe']),
        ('wochentag', OneHotEncoder(handle_unknown='ignore'), ['Wochentag'])
    ],
    remainder='drop'
)
# Ensure 'KielerWoche' is int after preprocessing
df['KielerWoche'] = df['KielerWoche'].astype(int)

#erste fünf Zeilen anzeigen
print(df.head())



Bewoelkung               70
Temperatur               16
Windgeschwindigkeit      16
Wettercode             2325
KielerWoche            9111
dtype: int64
        id      Datum  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
0  1307011 2013-07-01            1  148.828353         6.0     17.8375   
1  1307021 2013-07-02            1  159.793757         3.0     17.3125   
2  1307031 2013-07-03            1  111.885594         7.0     21.0750   
3  1307041 2013-07-04            1  168.864941         7.0     18.8500   
4  1307051 2013-07-05            1  171.280754         5.0     19.9750   

   Windgeschwindigkeit Wettercode  KielerWoche  feiertag  ...  Wochentag  \
0                 15.0       20.0            0         0  ...          0   
1                 10.0        nan            0         0  ...          1   
2                  6.0       61.0            0         0  ...          2   
3                  7.0       20.0            0         0  ...          3   
4                 12.0